In [ ]:
!git clone https://github.com/ruc-datalab/DeepAnalyze.git
%cd DeepAnalyze


Cloning into 'DeepAnalyze'...
remote: Enumerating objects: 2432, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 2432 (delta 172), reused 113 (delta 113), pack-reused 2177 (from 2)
Receiving objects: 100% (2432/2432), 23.27 MiB | 33.09 MiB/s, done.
Resolving deltas: 100% (595/595), done.
/content/DeepAnalyze


In [3]:
!pip install transformers==4.37.2 accelerate sentencepiece pandas torch



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 133.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.2
    Uninstalling tokenizers-0.22.2:
      Successfully uninstalled tokenizers-0.22.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.6
    Uninstalling transformers-4.57.6:
      Successfully uninstalled transformers-4.57.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.2.0 requires transformers<6.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.


In [4]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM


In [5]:
model_id = "Qwen/Qwen2.5-1.5B-Instruct"  # SAFE, CPU runnable

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cpu"
)

print("✅ Model Loaded Successfully")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

✅ Model Loaded Successfully


In [6]:
df = pd.read_csv("/content/retail_mart_final.csv")
df.head()


,order_id,order_datetime,store_id,region,city,product_id,category,subcategory,unit_price,quantity,discount,total_sales,profit,customer_id,customer_type,payment_method,online_order,stockout_flag,weekday,month
0,ORD100000,2024-10-16 03:01:00,STORE_70,Punjab,Multan,PROD_2424,Apparel,Men,34.93,1,0.00,34.93,6.89,CUST_87397,Returning,Cash,0,0,Wednesday,10
1,ORD100001,2023-08-12 07:32:00,STORE_70,Punjab,Faisalabad,PROD_7873,Grocery,Personal Care,22.23,1,0.20,17.78,3.20,CUST_38893,Returning,Debit Card,0,0,Saturday,8
2,ORD100002,2024-12-15 13:21:00,STORE_49,ICT,Islamabad,PROD_2584,Electronics,Mobiles,72.95,4,0.05,277.21,19.53,CUST_57052,Returning,Debit Card,0,0,Sunday,12
3,ORD100003,2024-07-03 03:59:00,STORE_47,Punjab,Rawalpindi,PROD_4150,Grocery,Personal Care,55.46,1,0.20,44.37,7.71,CUST_19116,New,Credit Card,1,0,Wednesday,7
4,ORD100004,2023-08-27 03:24:00,STORE_35,ICT,Islamabad,PROD_2169,Home & Living,Decor,20.32,1,0.05,19.30,2.34,CUST_89840,New,Credit Card,0,0,Sunday,8


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        35000 non-null  object 
 1   order_datetime  35000 non-null  object 
 2   store_id        35000 non-null  object 
 3   region          35000 non-null  object 
 4   city            35000 non-null  object 
 5   product_id      35000 non-null  object 
 6   category        35000 non-null  object 
 7   subcategory     35000 non-null  object 
 8   unit_price      35000 non-null  float64
 9   quantity        35000 non-null  int64  
 10  discount        35000 non-null  float64
 11  total_sales     35000 non-null  float64
 12  profit          35000 non-null  float64
 13  customer_id     35000 non-null  object 
 14  customer_type   35000 non-null  object 
 15  payment_method  35000 non-null  object 
 16  online_order    35000 non-null  int64  
 17  stockout_flag   35000 non-null 

In [8]:
dataset_context = f"""
Columns: {list(df.columns)}
Data Types: {df.dtypes}
Missing Values: {df.isnull().sum()}
"""

prompt = f"""
You are a DeepAnalyze data understanding assistant.

Dataset Information:
{dataset_context}

Task:
Explain the meaning of each column clearly.
"""

inputs = tokenizer(prompt, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.3
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))



You are a DeepAnalyze data understanding assistant.

Dataset Information:

Columns: ['order_id', 'order_datetime', 'store_id', 'region', 'city', 'product_id', 'category', 'subcategory', 'unit_price', 'quantity', 'discount', 'total_sales', 'profit', 'customer_id', 'customer_type', 'payment_method', 'online_order', 'stockout_flag', 'weekday', 'month']
Data Types: order_id           object
order_datetime     object
store_id           object
region             object
city               object
product_id         object
category           object
subcategory        object
unit_price        float64
quantity            int64
discount          float64
total_sales       float64
profit            float64
customer_id        object
customer_type      object
payment_method     object
online_order        int64
stockout_flag       int64
weekday            object
month               int64
dtype: object
Missing Values: order_id          0
order_datetime    0
store_id          0
region            0
city 